In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Benin Solar Data Analysis\n",
    "\n",
    "This notebook contains the exploratory data analysis of solar data from Benin."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Import required libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from scipy import stats\n",
    "import plotly.express as px\n",
    "from windrose import WindroseAxes\n",
    "\n",
    "# Set style for better visualizations\n",
    "plt.style.use('seaborn')\n",
    "sns.set_palette('husl')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Data Loading and Initial Inspection"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Load the dataset\n",
    "df = pd.read_csv('../data/benin_solar.csv')\n",
    "\n",
    "# Display basic information\n",
    "print(\"Dataset Shape:\", df.shape)\n",
    "print(\"\\nDataset Info:\")\n",
    "df.info()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Summary Statistics & Missing Value Report"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Summary statistics\n",
    "print(\"Summary Statistics:\")\n",
    "df.describe()\n",
    "\n",
    "# Missing values report\n",
    "missing_values = df.isna().sum()\n",
    "missing_percentages = (missing_values / len(df)) * 100\n",
    "\n",
    "print(\"\\nColumns with >5% missing values:\")\n",
    "print(missing_percentages[missing_percentages > 5])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Outlier Detection"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Calculate Z-scores for key variables\n",
    "key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']\n",
    "\n",
    "for col in key_columns:\n",
    "    df[f'{col}_zscore'] = stats.zscore(df[col], nan_policy='omit')\n",
    "    outliers = abs(df[f'{col}_zscore']) > 3\n",
    "    print(f\"Number of outliers in {col}: {sum(outliers)}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Time Series Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Convert timestamp to datetime\n",
    "df['Timestamp'] = pd.to_datetime(df['Timestamp'])\n",
    "\n",
    "# Plot time series\n",
    "fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n",
    "axes = axes.ravel()\n",
    "\n",
    "df.plot(x='Timestamp', y='GHI', ax=axes[0], title='GHI over Time')\n",
    "df.plot(x='Timestamp', y='DNI', ax=axes[1], title='DNI over Time')\n",
    "df.plot(x='Timestamp', y='DHI', ax=axes[2], title='DHI over Time')\n",
    "df.plot(x='Timestamp', y='Tamb', ax=axes[3], title='Ambient Temperature over Time')\n",
    "\n",
    "plt.tight_layout()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Cleaning Impact Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Compare ModA & ModB pre/post cleaning\n",
    "cleaning_impact = df.groupby('Cleaning').agg({\n",
    "    'ModA': 'mean',\n",
    "    'ModB': 'mean'\n",
    "}).plot(kind='bar', title='Average Module Performance by Cleaning Status')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Correlation Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Correlation heatmap\n",
    "correlation_vars = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB']\n",
    "correlation_matrix = df[correlation_vars].corr()\n",
    "\n",
    "plt.figure(figsize=(10, 8))\n",
    "sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Wind Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Create wind rose plot\n",
    "ax = WindroseAxes.from_ax()\n",
    "ax.bar(df['WD'], df['WS'], normed=True, opening=0.8, edgecolor='white')\n",
    "ax.set_legend()\n",
    "plt.title('Wind Rose Plot')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Temperature and Humidity Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Create bubble chart\n",
    "fig = px.scatter(df, x='Tamb', y='GHI', size='RH',\n",
    "                 title='Temperature vs GHI with Relative Humidity',\n",
    "                 labels={'Tamb': 'Ambient Temperature',\n",
    "                        'GHI': 'Global Horizontal Irradiance',\n",
    "                        'RH': 'Relative Humidity'})\n",
    "fig.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}